
# Mental Health Chatbot Training Notebook

This notebook provides a comprehensive pipeline to train a conversational mental health assistant. 
The system integrates emotion classification using `SamLowe/roberta-base-go_emotions` and text generation using `T5`.
It processes multiple cleaned datasets, performs training, evaluation, and finally builds a chatbot interface using Gradio or Streamlit.

## Objectives

- Load and preprocess multiple mental health-related datasets into a consistent question/answer format
- Simulate and encode multi-label emotion annotations using `MultiLabelBinarizer`
- Train a RoBERTa-based emotion classification model with live metric logging (accuracy, F1, precision, recall)
- Fine-tune two separate T5 models:
  - One for emotionally guided chatbot response generation
  - One for direct factual Q&A answering
- Implement emotion-aware routing logic that selects the appropriate model at inference time
- Create a unified RoBERTa + T5 pipeline for real-time response generation
- Build a Gradio chatbot interface using the full system
- Evaluate all models with live logging and inference testing
- Save all models and tokenizers in `./saved_models/`
- Save a `.pt` metadata file pointing to model paths for easy deployment


In [1]:
# Required libraries

import pandas as pd
# Data Handling
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

# PyTorch & Transformers
import torch
import torch.nn.functional as F
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    T5Tokenizer,
    T5ForConditionalGeneration,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer
)

from torchvision.ops import nms
import shutil
# Hugging Face Datasets & Evaluation
from datasets import load_dataset, Dataset, concatenate_datasets
from evaluate import load as load_metric

# External Evaluation Libraries
import bert_score
import rouge_score

# Gradio for UI
import gradio as gr

# Progress Bar
from tqdm.auto import tqdm

# Optional for advanced memory management (not required unless doing lazy loading)
from accelerate import init_empty_weights



In [2]:
device = torch.device("cuda:0")
print("GPU Available:", torch.cuda.is_available())  # True if a GPU is accessible
print("Current Device Index:", torch.cuda.current_device())  # e.g., 0
print("Device Name:", torch.cuda.get_device_name(0))  # e.g., "NVIDIA GeForce RTX 3070"

# Example: set default tensor type to GPU-based FloatTensor (optional)
# This will make ALL newly created tensors go to GPU (Float32).
# torch.set_default_dtype(torch.float32)
# torch.set_default_tensor_type(torch.cuda.FloatTensor)

GPU Available: True
Current Device Index: 0
Device Name: NVIDIA GeForce RTX 3070 Laptop GPU


## Load and Preprocess Datasets

In [3]:

import pandas as pd
from datasets import Dataset, concatenate_datasets

# File paths
dataset_paths = {
    "ds1": "./data/ds1_transformed_mental_health_chatbot_dataset.csv",
    "ds2": "./data/ds2_transformed_mental_health_chatbot.csv",
    "ds3": "./data/ds3_mental_health_faq_cleaned.csv",
    "ds4": "./data/ds4_mental_health_chatbot_dataset_merged_modes.csv",
    "ds5": "./data/ds5_Mental_Health_FAQ.csv",
    "ds6": "./data/ds6_mental_health_counseling.csv"
}

# Enable/disable datasets
dataset_switches = {
    "ds1": True,
    "ds2": False,
    "ds3": False,
    "ds4": False,
    "ds5": False,
    "ds6": False
}

# Clean function
def load_and_clean_csv(path):
    df = pd.read_csv(path)
    df.columns = [col.lower().strip() for col in df.columns]
    if "prompt" in df.columns and "response" in df.columns:
        df = df.rename(columns={"prompt": "question", "response": "answer"})
    if "questions" in df.columns:
        df = df.rename(columns={"questions": "question"})
    if "answers" in df.columns:
        df = df.rename(columns={"answers": "answer"})
    df = df[[col for col in df.columns if col in ["question", "answer"]]]
    df.dropna(inplace=True)
    return Dataset.from_pandas(df)

# Load selected datasets
datasets_list = []
for name, path in dataset_paths.items():
    if dataset_switches[name]:
        datasets_list.append(load_and_clean_csv(path))

# Merge and split
combined_dataset = concatenate_datasets(datasets_list).shuffle(seed=42)
split_dataset = combined_dataset.train_test_split(test_size=0.1)

train_ds = split_dataset["train"]
test_ds = split_dataset["test"]

train_ds, test_ds


(Dataset({
     features: ['question', 'answer'],
     num_rows: 154
 }),
 Dataset({
     features: ['question', 'answer'],
     num_rows: 18
 }))

## Prepare 'labels' Column with MultiLabelBinarizer

In [4]:

from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset

# Convert to DataFrame
df_train = train_ds.to_pandas()
df_test = test_ds.to_pandas()

# Simulated emotion annotations (replace with real data if available)
import random
emotions_list = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
                 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
                 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization',
                 'relief', 'remorse', 'sadness', 'surprise']
df_train["emotions"] = [random.sample(emotions_list, k=random.randint(1, 3)) for _ in range(len(df_train))]
df_test["emotions"] = [random.sample(emotions_list, k=random.randint(1, 3)) for _ in range(len(df_test))]

# Encode with MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=emotions_list)
df_train["labels"] = list(mlb.fit_transform(df_train["emotions"]))
df_test["labels"] = list(mlb.transform(df_test["emotions"]))

# Convert back to Hugging Face Dataset
train_ds = Dataset.from_pandas(df_train)
test_ds = Dataset.from_pandas(df_test)


## Configure RoBERTa for Multi-Label Classification

In [5]:

from transformers import RobertaConfig, RobertaForSequenceClassification

# Explicitly define configuration for multi-label classification
config = RobertaConfig.from_pretrained(
    "SamLowe/roberta-base-go_emotions",
    problem_type="multi_label_classification"
)

# Load model with custom config
model_emo = RobertaForSequenceClassification.from_pretrained(
    "SamLowe/roberta-base-go_emotions",
    config=config
)


## Train RoBERTa with Correct Labels

In [ ]:
import os
from transformers import (
    RobertaTokenizer, RobertaForSequenceClassification,
    Trainer, TrainingArguments, RobertaConfig,
    DataCollatorWithPadding
)
from datasets import Features, Value, Sequence
import numpy as np
import torch

# Model Save Directory
model_path = "./saved_models/emotion_classifier"


# Load tokenizer
tokenizer_emo = RobertaTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")

# Load existing model if found, otherwise initialize from base
if os.path.exists(model_path):
    print("Loading previously trained model...")
    config = RobertaConfig.from_pretrained(model_path)
    model_emo = RobertaForSequenceClassification.from_pretrained(model_path, config=config)
else:
    print("No existing model found. Starting fresh.")
    config = RobertaConfig.from_pretrained("SamLowe/roberta-base-go_emotions", problem_type="multi_label_classification")
    model_emo = RobertaForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions", config=config)

# Define label format
features = Features({
    "question": Value("string"),
    "answer": Value("string"),
    "emotions": Sequence(Value("string")),
    "labels": Sequence(Value("float32")) 
})

# Apply format to datasets
train_ds = train_ds.cast(features)
test_ds = test_ds.cast(features)

# Tokenization function
def tokenize_emotion(example):
    enc = tokenizer_emo(example["question"], padding="max_length", truncation=True)
    enc["labels"] = example["labels"]
    return enc

# Tokenize
train_emo = train_ds.map(tokenize_emotion, batched=False)
test_emo = test_ds.map(tokenize_emotion, batched=False)

train_emo.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_emo.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

collator = DataCollatorWithPadding(tokenizer=tokenizer_emo, return_tensors="pt")

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs > 0.5).int().numpy()

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro', zero_division=0)
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }



# Training arguments
training_args_emo = TrainingArguments(
    output_dir=model_path,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",        
    logging_steps=10,                
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    load_best_model_at_end=True,
    overwrite_output_dir=False, 
    report_to="none"
)

from transformers import Trainer

trainer_emo = Trainer(
    model=model_emo,
    args=training_args_emo,
    train_dataset=train_emo,
    eval_dataset=test_emo,
    tokenizer=tokenizer_emo,
    data_collator=collator,
    compute_metrics=compute_metrics
)

trainer_emo.train()
trainer_emo.save_model(model_path)


# Trainer setup
trainer_emo = Trainer(
    model=model_emo,
    args=training_args_emo,
    train_dataset=train_emo,
    eval_dataset=test_emo,
    tokenizer=tokenizer_emo,
    data_collator=collator
)

# Training
trainer_emo.train()

# Final save (no overwrite here, path already protected)
trainer_emo.save_model(model_path)


Loading previously trained model...


Casting the dataset:   0%|          | 0/154 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/18 [00:00<?, ? examples/s]

Map:   0%|          | 0/154 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

c:\Users\mward\anaconda3\envs\dev2\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\mward\AppData\Local\Temp\ipykernel_4056\1844043087.py:91: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_emo = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.261600,0.273434,0.000000,0.000000,0.000000,0.000000
2,0.271400,0.272481,0.000000,0.000000,0.000000,0.000000


## Train T5 for Response Generation

In [ ]:
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Define model save path
t5_response_path = "./saved_models/t5_response_generator"

# Delete invalid or empty model folder if necessary
if os.path.exists(t5_response_path) and not any(
    fname.endswith((".bin", ".safetensors", ".h5", ".index", ".msgpack")) for fname in os.listdir(t5_response_path)
):
    print("Found empty or invalid model folder. Deleting it...")
    shutil.rmtree(t5_response_path)

# Load tokenizer
tokenizer_t5 = T5Tokenizer.from_pretrained("t5-small")

# Load model safely
if os.path.exists(t5_response_path) and os.path.exists(os.path.join(t5_response_path, "pytorch_model.bin")):
    print("Loading previously trained T5 response model...")
    model_t5_response = T5ForConditionalGeneration.from_pretrained(t5_response_path)
else:
    print("No valid saved model found. Loading 't5-small' fresh.")
    model_t5_response = T5ForConditionalGeneration.from_pretrained("t5-small")

def tokenize_t5_chat(example):
    input_text = "chat: " + example["question"]
    target_text = example["answer"]

    model_inputs = tokenizer_t5(
        input_text,
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    with tokenizer_t5.as_target_tokenizer():
        labels = tokenizer_t5(
            target_text,
            max_length=128,
            truncation=True,
            padding="max_length"
        )["input_ids"]

    # Ensure data returned as plain Python int lists
    return {
        "input_ids": [int(i) for i in model_inputs["input_ids"]],
        "attention_mask": [int(i) for i in model_inputs["attention_mask"]],
        "labels": [int(i) for i in labels]
    }



train_chat = train_ds.map(tokenize_t5_chat)
test_chat = test_ds.map(tokenize_t5_chat)

train_chat.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_chat.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])



from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs > 0.5).int().numpy()

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro', zero_division=0)
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }


# Training arguments
training_args_chat = TrainingArguments(
    output_dir=t5_response_path,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",        
    logging_steps=10,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    load_best_model_at_end=True,
    overwrite_output_dir=False,  # <--- Prevent overwriting!
    report_to="none"
)

from transformers import Trainer

trainer_chat = Trainer(
    model=model_t5_response,
    args=training_args_chat,
    train_dataset=train_chat,
    eval_dataset=test_chat,
    tokenizer=tokenizer_t5,
    compute_metrics=compute_metrics 
)

trainer_chat.train()
trainer_chat.save_model(t5_response_path)



# Trainer
trainer_chat = Trainer(
    model=model_t5_response,
    args=training_args_chat,
    train_dataset=train_chat,
    eval_dataset=test_chat,
    tokenizer=tokenizer_t5,
    compute_metrics=compute_metrics
)

# Train and Save
trainer_chat.train()
trainer_chat.save_model(t5_response_path)


Found empty or invalid model folder. Deleting it...
No valid saved model found. Loading 't5-small' fresh.


Map:   0%|          | 0/154 [00:00<?, ? examples/s]

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

C:\Users\mward\AppData\Local\Temp\ipykernel_25344\3013294863.py:96: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_chat = Trainer(


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

## Train T5 for Q&A Assistant

In [ ]:
import os
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Define model save path
t5_qa_path = "./saved_models/t5_qa"

# Load existing model or initialize new
if os.path.exists(t5_qa_path):
    print("🔁 Loading previously trained T5 Q&A model...")
    model_t5_qa = T5ForConditionalGeneration.from_pretrained(t5_qa_path)
else:
    print("🆕 No existing Q&A model found. Starting from scratch.")
    model_t5_qa = T5ForConditionalGeneration.from_pretrained("t5-small")

# Tokenization function
def tokenize_t5_qa(example):
    input_text = "question: " + example["question"]
    target_text = example["answer"]
    model_inputs = tokenizer_t5(input_text, max_length=128, truncation=True, padding="max_length")
    with tokenizer_t5.as_target_tokenizer():
        labels = tokenizer_t5(target_text, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize datasets
train_qa = train_ds.map(tokenize_t5_qa, batched=True)
test_qa = test_ds.map(tokenize_t5_qa, batched=True)

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs > 0.5).int().numpy()

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro', zero_division=0)
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }


# Training arguments
training_args_qa = TrainingArguments(
    output_dir=t5_qa_path,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",        
    logging_steps=10,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    load_best_model_at_end=True,
    overwrite_output_dir=False,  # <--- Protects against overwrite
    report_to="none",
    compute_metrics=compute_metrics
    
)

# Trainer setup
trainer_qa = Trainer(
    model=model_t5_qa,
    args=training_args_qa,
    train_dataset=train_qa,
    eval_dataset=test_qa,
    tokenizer=tokenizer_t5,
    compute_metrics=compute_metrics
)

# Train and save
trainer_qa.train()
trainer_qa.save_model(t5_qa_path)

# Trainer setup
trainer_qa = Trainer(
    model=model_t5_qa,
    args=training_args_qa,
    train_dataset=train_qa,
    eval_dataset=test_qa,
    tokenizer=tokenizer_t5
)

# Train and save
trainer_qa.train()
trainer_qa.save_model(t5_qa_path)


## Unified Emotion-Aware Response System (RoBERTa + T5 Routing Logic)

In [ ]:
from transformers import (
    RobertaForSequenceClassification, RobertaTokenizer,
    T5ForConditionalGeneration, T5Tokenizer
)
import torch
import torch.nn.functional as F

# Load models and tokenizers
emotion_model = RobertaForSequenceClassification.from_pretrained("./saved_models/emotion_classifier")
emotion_tokenizer = RobertaTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")

t5_chat_model = T5ForConditionalGeneration.from_pretrained("./saved_models/t5_response_generator")
t5_qa_model = T5ForConditionalGeneration.from_pretrained("./saved_models/t5_qa")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Emotion labels from GoEmotions dataset
emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
    'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
    'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise'
]

# Emotion classifier
def detect_emotions(text, threshold=0.5):
    emotion_model.eval()
    inputs = emotion_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = emotion_model(**inputs).logits
        probs = torch.sigmoid(logits).squeeze().tolist()
    detected = [(emotion_labels[i], p) for i, p in enumerate(probs) if p > threshold]
    return [label for label, _ in detected]

# Response generator with dynamic routing
def generate_combined_response(user_input):
    emotions = detect_emotions(user_input)

    emotional_keywords = {
        'joy', 'sadness', 'anger', 'fear', 'love', 'grief', 'remorse', 'disappointment', 'gratitude', 'caring'
    }
    use_chat_model = any(e in emotional_keywords for e in emotions)

    if use_chat_model:
        prefix = "chat: "
        model = t5_chat_model
    else:
        prefix = "question: "
        model = t5_qa_model

    context = f"{' '.join(emotions)}: {user_input}" if emotions else user_input
    input_text = prefix + context

    inputs = t5_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output_ids = model.generate(inputs["input_ids"], max_length=128)
    response = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return {
        "Detected Emotions": emotions,
        "Model Used": "Chat Model" if use_chat_model else "QA Model",
        "Response": response
    }

# 🔍 Test it
generate_combined_response("I feel really hopeless and angry all the time.")


## Save All Final Models and Tokenizers (RoBERTa + T5s)

In [ ]:

# Save all models and tokenizers using Hugging Face's save_pretrained
from transformers import RobertaForSequenceClassification, RobertaTokenizer, T5Tokenizer

# Load models
chat_model = T5ForConditionalGeneration.from_pretrained("./saved_models/t5_response_generator")
qa_model = T5ForConditionalGeneration.from_pretrained("./saved_models/t5_qa")
emotion_model = RobertaForSequenceClassification.from_pretrained("./saved_models/emotion_classifier")

# Load tokenizers
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
emotion_tokenizer = RobertaTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")

# Save full models and tokenizers
chat_model.save_pretrained("./saved_models/final/chat_model")
qa_model.save_pretrained("./saved_models/final/qa_model")
emotion_model.save_pretrained("./saved_models/final/emotion_model")

t5_tokenizer.save_pretrained("./saved_models/final/t5_tokenizer")
emotion_tokenizer.save_pretrained("./saved_models/final/emotion_tokenizer")


## Save Final Model Metadata (.pt) for Inference Pipeline

In [ ]:
# Lightweight metadata dictionary (pointer-style)
final_model_metadata = {
    "chat_model_path": "./saved_models/final/chat_model",
    "qa_model_path": "./saved_models/final/qa_model",
    "emotion_model_path": "./saved_models/final/emotion_model",
    "t5_tokenizer_path": "./saved_models/final/t5_tokenizer",
    "emotion_tokenizer_path": "./saved_models/final/emotion_tokenizer",
    "labels": emotion_labels  # list of emotion label names only
}

# Save just this dictionary (no models inside)
torch.save(final_model_metadata, "./saved_models/final_model_metadata.pt")


## Gradio Chatbot Interface

In [ ]:
import gradio as gr

chat_history = []

def gradio_chat_interface(user_input):
    global chat_history
    response_data = generate_combined_response(user_input)
    chatbot_response = response_data['Response']
    chat_history.append(("You: " + user_input, "Bot: " + chatbot_response))
    return chat_history

interface = gr.Interface(
    fn=gradio_chat_interface,
    inputs="text",
    outputs="chatbot",
    title="Mental Health Chatbot",
    description="Emotion-aware chatbot using RoBERTa + T5",
    live=True
)

interface.launch()


## Model Evaluation Metrics and Sample Inference

In [ ]:

import evaluate
import numpy as np

# Load metrics
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")
accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")
f1 = evaluate.load("f1")

# Prepare sample data
sample_size = min(100, len(test_ds))
sample = test_ds.select(range(sample_size))

def generate_predictions(model, tokenizer, dataset):
    inputs = [f"question: {x['question']}" for x in dataset]
    inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output_ids = model.generate(inputs['input_ids'], max_length=128)
    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    refs = [x["answer"] for x in dataset]
    return preds, refs

predictions, references = generate_predictions(model_t5_qa, tokenizer_t5, sample)

# Evaluate
rouge_results = rouge.compute(predictions=predictions, references=references)
bertscore_results = bertscore.compute(predictions=predictions, references=references, lang="en")

# Binary approximation for classification scores
true_labels = [1 if r.strip().lower() in p.strip().lower() else 0 for p, r in zip(predictions, references)]
accuracy_result = accuracy.compute(references=true_labels, predictions=true_labels)
precision_result = precision.compute(references=true_labels, predictions=true_labels)
recall_result = recall.compute(references=true_labels, predictions=true_labels)
f1_result = f1.compute(references=true_labels, predictions=true_labels)

# Display
print("ROUGE Scores:", rouge_results)
print("BERTScore:", bertscore_results)
print("Accuracy:", accuracy_result)
print("Precision:", precision_result)
print("Recall:", recall_result)
print("F1 Score:", f1_result)

# RoBERTa Emotion Classifier - Inference Test
sample_input = "I feel like I'm breaking down and can't handle anything."

inputs = tokenizer_emo(sample_input, return_tensors="pt", truncation=True, padding=True)
model_emo.eval()
with torch.no_grad():
    logits = model_emo(**inputs).logits
probs = torch.sigmoid(logits).squeeze().tolist()

predicted_emotions = [emotion_labels[i] for i, p in enumerate(probs) if p > 0.5]
print("🔍 Detected Emotions:", predicted_emotions)

# T5 QA Model - Inference Test
qa_input = "question: What are some ways to manage daily anxiety?"
inputs = tokenizer_t5(qa_input, return_tensors="pt")

qa_model.eval()
with torch.no_grad():
    output_ids = qa_model.generate(inputs["input_ids"], max_length=128)

response = tokenizer_t5.decode(output_ids[0], skip_special_tokens=True)
print("💡 QA Model Response:", response)

# T5 QA Model - Inference Test
qa_input = "question: What are some ways to manage daily anxiety?"
inputs = tokenizer_t5(qa_input, return_tensors="pt")

qa_model.eval()
with torch.no_grad():
    output_ids = qa_model.generate(inputs["input_ids"], max_length=128)

response = tokenizer_t5.decode(output_ids[0], skip_special_tokens=True)
print("💡 QA Model Response:", response)



In [ ]:
print(emotion_labels[:5])